In [ ]:
import pandas as pd
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('submission.csv')
df

In [ ]:
def fill_anomalies(df, window_size=4, threshold=2, loops=2):
    count = 0
    for l in range(loops):
        for index, row in df.iterrows():
            if row['anomaly'] == 0:
                start_index = max(index - window_size, 0)
                end_index = min(index + window_size + 1, len(df))
                window = df['anomaly'][start_index:end_index]
                
                # Prüfe, ob mindestens eine '1' im Bereich vor und nach der '0' ist
                if 1 in df['anomaly'][start_index:index].values and 1 in df['anomaly'][index + 1:end_index].values:
                    window = df['anomaly'][start_index:end_index]
                    if window.sum() >= threshold:
                        df.at[index, 'anomaly'] = 1
                        count += 1
    print('Gefüllt:', count)

    return df

def remove_anomalies(df, window_size=5, threshold=1):
    count = 0
    for index, row in df.iterrows():
        if row['anomaly'] == 1:
            start_index = max(index - window_size, 0)
            end_index = min(index + window_size + 1, len(df))
            
            if 0 in df['anomaly'][start_index:index].values and 0 in df['anomaly'][index + 1:end_index].values:
                window = df['anomaly'][start_index:end_index]
                if window.sum() <= threshold:
                    df.at[index, 'anomaly'] = 0
                    count += 1
    print('Entfernt:', count)

    return df

# Anwenden der Funktion


In [ ]:
df_filled = fill_anomalies(df.copy(), window_size=4, threshold=4, loops=2)

df_filled_removed = remove_anomalies(df_filled.copy(), window_size=5, threshold=4)


In [ ]:

# Erstellen der Subplots
parts = 38*2
teil_groesse = len(df_filled_removed) // parts
fig, axes = plt.subplots(parts, 3, figsize=(20, 1*parts))

for i in range(parts):
    # Auswahl des entsprechenden Teils der DataFrames
    teil_df = df.iloc[i * teil_groesse:(i + 1) * teil_groesse]
    teil_df_filled = df_filled.iloc[i * teil_groesse:(i + 1) * teil_groesse]
    teil_df_filled_removed = df_filled_removed.iloc[i * teil_groesse:(i + 1) * teil_groesse]

    # Erstellen der Balkendiagramme für den aktuellen Teil
    axes[i, 0].bar(teil_df['id'], teil_df['anomaly'], width=1.0)
    
    axes[i, 1].bar(teil_df_filled['id'], teil_df_filled['anomaly'], width=1.0)

    axes[i, 2].bar(teil_df_filled_removed['id'], teil_df_filled_removed['anomaly'], width=1.0)
    axes[i, 0].set_ylabel('Part: ' + str(i))
    # Setzen der Achsenbeschriftungen für alle Spalten
    # for j in range(3):
    #     axes[i, j].set_xlabel('ID')
    #     axes[i, j].set_ylabel('Anomalie (1 oder 0)')
    #     axes[i, j].set_xlim(i * teil_groesse, (i + 1) * teil_groesse)

axes[0, 0].set_title(f'Original Daten Teil {i+1}')
axes[0, 1].set_title(f'Gefüllte Daten Teil {i+1}')
axes[0, 2].set_title(f'Gefüllte & Entfernte Daten Teil {i+1}')


# axes[0, 0].set_xlabel('ID')
# axes[0, 1].set_ylabel('Anomalie (1 oder 0)')
# axes[0, 2].set_xlim(i * teil_groesse, (i + 1) * teil_groesse)

# Anpassung des Layouts
plt.tight_layout()

# Anzeigen der Plots
plt.show()


In [ ]:
df_filled.to_csv('submission_one_step.csv', index=False)
df_filled_removed.to_csv('submission_post.csv', index=False)